<a href="https://colab.research.google.com/github/Taichi2005/GoogleColab_Whisper/blob/main/%E9%AB%98%E6%80%A7%E8%83%BDURL%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97%E5%AE%9F%E8%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#### **セル 1: 環境のセットアップ (初回のみ実行)**
#@title 1. 🚀 環境のセットアップ (初回のみ実行)
#@markdown このセルを実行して、文字起こしに必要なライブラリとツールをインストールします。<br>
#@markdown 完了までに数分かかることがあります。

print("--- 1. yt-dlp (動画ダウンロードツール) をインストール中...")
!pip install -U yt-dlp &> /dev/null

print("--- 2. ffmpeg (メディア変換ツール) をインストール中...")
!apt-get update -qq && apt-get install -y -qq ffmpeg

print("--- 3. faster-whisper (高性能文字起こしエンジン) と関連ライブラリをインストール中...")
!pip install "faster-whisper>0.10.0" transformers torch &> /dev/null

print("\n✅ 全てのセットアップが完了しました。次のセルに進んでください。")

--- 1. yt-dlp (動画ダウンロードツール) をインストール中...
--- 2. ffmpeg (メディア変換ツール) をインストール中...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
--- 3. faster-whisper (高性能文字起こしエンジン) と関連ライブラリをインストール中...

✅ 全てのセットアップが完了しました。次のセルに進んでください。


In [2]:
#### **セル 2: URLを指定して文字起こしを実行**

#@title 2. 🎬 高性能URL文字起こし実行セル
#@markdown ### 1. 動画のURL
#@markdown ---
#@markdown 文字起こしをしたい動画のURLを貼り付けてください。（YouTube, Vimeoなど）
video_url = "https://www.youtube.com/watch?v=sln-2o_yL9E" #@param {type:"string"}

#@markdown ### 2. モデルとパフォーマンス設定
#@markdown ---
#@markdown **モデル**: `deepdml/faster-whisper-large-v3-turbo-ct2`は、`large-v3`を高速化した推奨モデルです。<br>
#@markdown **計算タイプ**: `float16`は精度をほぼ維持しつつ高速化する推奨設定。`int8`はさらに高速・省メモリになります。
model_name = "deepdml/faster-whisper-large-v3-turbo-ct2" #@param ["deepdml/faster-whisper-large-v3-turbo-ct2", "large-v3", "large-v2", "distil-large-v3", "medium", "small", "base"]
compute_type = "float16" #@param ["float16", "int8"]

#@markdown ### 3. VAD (音声区間検出) 設定
#@markdown ---
#@markdown VADを有効にすると、無音区間を自動で除去し、精度と速度を向上させることができます。
use_vad_filter = True #@param {type:"boolean"}
#@markdown ここで指定したミリ秒以上の無音を「発話の区切り」とみなします。
vad_min_silence_duration_ms = 500 #@param {type:"slider", min:100, max:2000, step:100}

#@markdown ### 4. 保存先の設定
#@markdown ---
#@markdown 文字起こし結果のテキストファイル（.txt）を保存するGoogle Driveのフォルダを指定します。
drive_transcript_output_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts" #@param {type:"string"}


# --- ここから下は実行コード（編集不要） ---
import os
import re
import yt_dlp
from faster_whisper import WhisperModel
import torch
from datetime import datetime, timezone, timedelta
import time
from google.colab import drive

# --- ヘルパー関数 ---
def get_current_timestamp():
    """現在時刻を[YYYY-MM-DD HH:MM:SS]形式の文字列で返す"""
    JST = timezone(timedelta(hours=+9))
    return datetime.now(JST).strftime('%Y-%m-%d %H:%M:%S')

def sanitize_filename(filename):
    """ファイル名として不適切な文字を全角に置換する"""
    return re.sub(r'[\\/*?:"<>|]', lambda m: {'\\': '＼', '/': '／', '*': '＊', '?': '？', ':': '：', '"': '”', '<': '＜', '>': '＞', '|': '｜'}[m.group(0)], filename)

# --- パイプライン本体 ---
start_pipeline_time = time.time()
print(f"{get_current_timestamp()} --- 1. 環境設定とディレクトリの準備 ---")

# Google Driveのマウントと保存先ディレクトリの準備
try:
    drive.mount('/content/drive', force_remount=True)
    os.makedirs(drive_transcript_output_dir, exist_ok=True)
    print(f"✅ Google Driveマウント成功。出力先: {drive_transcript_output_dir}")
except Exception as e:
    print(f"⚠️ Google Driveのマウントに失敗しました: {e}")
    drive_transcript_output_dir = "/content/transcripts"
    os.makedirs(drive_transcript_output_dir, exist_ok=True)
    print(f"結果はColabの一時フォルダ '{drive_transcript_output_dir}' に保存されます。")

temp_audio_dir = "/content/temp_audio"
os.makedirs(temp_audio_dir, exist_ok=True)

# --- モデルのロード ---
print(f"\n{get_current_timestamp()} --- 2. モデル '{model_name}' ({compute_type}) をロード中... ---")
model = None
try:
    start_load_time = time.time()
    model = WhisperModel(model_name, device="cuda" if torch.cuda.is_available() else "cpu", compute_type=compute_type)
    end_load_time = time.time()
    print(f"✅ モデルのロードが完了しました。({end_load_time - start_load_time:.2f}秒)")
except Exception as e:
    print(f"💥 モデルのロード中に致命的なエラーが発生しました: {e}")
    print("VRAMが不足している可能性があります。モデルサイズを小さくするか、Colabのランタイムを再起動してください。")

# --- メイン処理 ---
if model and video_url:
    print(f"\n{get_current_timestamp()} --- 3. メイン処理開始 ---")
    start_process_time = time.time()

    # yt-dlpによる音声抽出
    print(f"  - URLから音声を抽出中... (対象: {video_url})")
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3', 'preferredquality': '192'}],
        'outtmpl': os.path.join(temp_audio_dir, '%(id)s.%(ext)s'),
        'quiet': True, 'noprogress': True,
    }

    audio_file_path = None
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            video_info = ydl.extract_info(video_url, download=True)
            video_id = video_info.get('id', 'temp_video')
            video_title = video_info.get('title', '無題の動画')

            # 変換後のファイルパスを特定
            base_path = os.path.join(temp_audio_dir, video_id)
            audio_file_path = f"{base_path}.mp3"
            if not os.path.exists(audio_file_path):
                 webm_path = f"{base_path}.webm"
                 if os.path.exists(webm_path): os.rename(webm_path, audio_file_path)
                 else: raise FileNotFoundError("ダウンロードされた音声ファイルが見つかりません。")
        print(f"  - 音声抽出完了: {os.path.basename(audio_file_path)}")
    except Exception as e:
        print(f"  - 💥 音声のダウンロードまたは変換中にエラーが発生しました: {e}")

    # 文字起こし実行
    if audio_file_path and os.path.exists(audio_file_path):
        try:
            print(f"  - 文字起こしを実行中... (VAD: {'有効' if use_vad_filter else '無効'})")
            segments, info = model.transcribe(
                audio_file_path, beam_size=5,
                vad_filter=use_vad_filter,
                vad_parameters=dict(min_silence_duration_ms=vad_min_silence_duration_ms)
            )
            print(f"  - 検出言語: {info.language} (確率: {info.language_probability:.2f})")

            # 結果をタイムスタンプ付きテキストとして保存
            sanitized_title = sanitize_filename(video_title)
            output_txt_filename = f"{sanitized_title}.txt"
            drive_output_path = os.path.join(drive_transcript_output_dir, output_txt_filename)

            with open(drive_output_path, "w", encoding="utf-8") as f:
                # 以前のスクリプトには無かった便利なメタ情報を追加
                f.write(f"■ 動画タイトル: {video_title}\n")
                f.write(f"■ 動画URL: {video_url}\n")
                f.write(f"■ 文字起こしモデル: {model_name} (計算タイプ: {compute_type})\n")
                f.write(f"■ 処理日時: {get_current_timestamp()}\n\n---\n\n")

                # 「完璧だった」スクリプトのタイムスタンプ形式を再現
                for segment in segments:
                    line = f"[{segment.start:0>7.2f}s -> {segment.end:0>7.2f}s] {segment.text.strip()}\n"
                    f.write(line)

            print(f"  - 📄 結果を保存しました: {drive_output_path}")

        except Exception as e:
            print(f"  - 💥 文字起こし中にエラーが発生しました: {e}")
        finally:
            # ローカルの一時ファイルをクリーンアップ
            os.remove(audio_file_path)

    end_process_time = time.time()
    print(f"\n{get_current_timestamp()} ■ 処理完了 ({end_process_time - start_process_time:.2f}秒)")

elif not video_url:
    print("⚠️ 動画のURLが入力されていません。URLを指定して再度実行してください。")

if model is not None:
    del model # メモリを解放

print(f"\n{get_current_timestamp()} --- 🎉 全ての処理が完了しました ---")

2025-10-25 22:18:01 --- 1. 環境設定とディレクトリの準備 ---
Mounted at /content/drive
✅ Google Driveマウント成功。出力先: /content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts

2025-10-25 22:18:26 --- 2. モデル 'deepdml/faster-whisper-large-v3-turbo-ct2' (float16) をロード中... ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

vocabulary.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

💥 モデルのロード中に致命的なエラーが発生しました: Requested float16 compute type, but the target device or backend do not support efficient float16 computation.
VRAMが不足している可能性があります。モデルサイズを小さくするか、Colabのランタイムを再起動してください。

2025-10-25 22:18:56 --- 🎉 全ての処理が完了しました ---
